In [1]:
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np 
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
import optuna
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
df=pd.read_csv('cleaned.csv',index_col=0)
testset=pd.read_csv('test.csv',index_col=0)

In [3]:
X = df.drop('SMOKING', axis=1)
y = df['SMOKING']

X_test = testset.drop('SMOKING', axis=1)
y_test = testset['SMOKING']

In [4]:
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import numpy as np
import xgboost as xgb


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def objective(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 50, 300, step=10)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.3, log=True)

    scores = []
    F1=[]

    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        
        # scaler = MinMaxScaler()
        # X_train_scaled = scaler.fit_transform(X_train)
        # X_val_scaled = scaler.transform(X_val)

    
        model = xgb.XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            n_jobs=-1,
            random_state=42
        )

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)

        acc = accuracy_score(y_val, y_pred)
        
        F1c=f1_score(y_val, y_pred, average='macro') 
        F1.append(F1c)
        

        y_pred_train = model.predict(X_train)
        acc_train = accuracy_score(y_train, y_pred_train)
        f1_train = f1_score(y_train, y_pred_train, average='macro')
        

        overfit_penalty = abs(acc_train - acc)
        score = 0.5 * acc - 0.5 * overfit_penalty
        
        
        
        scores.append(score)

    return np.mean(scores)


sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='maximize',sampler=sampler)

study.optimize(objective, n_trials=20)  


print("best_params:", study.best_params)
print(f"best_value: {study.best_value:.4f}")

[I 2025-08-16 09:39:03,214] A new study created in memory with name: no-name-18a596d5-4c3a-4729-906b-988d580c921e
[I 2025-08-16 09:39:03,392] Trial 0 finished with value: 0.3193927025355597 and parameters: {'n_estimators': 140, 'max_depth': 10, 'learning_rate': 0.1001303991139125}. Best is trial 0 with value: 0.3193927025355597.
[I 2025-08-16 09:39:03,535] Trial 1 finished with value: 0.26337024654501084 and parameters: {'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.009470040922904443}. Best is trial 0 with value: 0.3193927025355597.
[I 2025-08-16 09:39:03,632] Trial 2 finished with value: 0.32858575394470974 and parameters: {'n_estimators': 60, 'max_depth': 9, 'learning_rate': 0.05859268690985102}. Best is trial 2 with value: 0.32858575394470974.
[I 2025-08-16 09:39:03,757] Trial 3 finished with value: 0.2552858011378679 and parameters: {'n_estimators': 230, 'max_depth': 2, 'learning_rate': 0.2652261985899885}. Best is trial 2 with value: 0.32858575394470974.
[I 2025-08-16 0

best_params: {'n_estimators': 80, 'max_depth': 5, 'learning_rate': 0.03983915420249222}
best_value: 0.3415


In [5]:

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []

for train_idx, val_idx in skf.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = xgb.XGBClassifier(
            objective='binary:logistic',
            max_depth=5,
            learning_rate=0.03983915420249222,
            n_estimators=80,
            eval_metric='logloss',
            random_state=42,
            # min_child_weight=5,         
            subsample=0.8,             
            # # colsample_bytree=0.7,       
            
            n_jobs=-1
        )
    model.fit(X_train, y_train)  

    y_test_pred = model.predict(X_val)

    acc_test = accuracy_score(y_val, y_test_pred)
    precision_test = precision_score(y_val, y_test_pred, average='macro')
    recall_test = recall_score(y_val, y_test_pred, average='macro')
    f1_test = f1_score(y_val, y_test_pred, average='macro')

    train_pred = model.predict(X_train)  
    train_acc = accuracy_score(y_train, train_pred)
    print(f"Train Accuracy: {train_acc:.4f}, test Accuracy: {acc_test:.4f}")

    print(f"test Accuracy:  {acc_test:.4f}")
    print(f"test Precision: {precision_test:.4f}")
    print(f"test Recall:    {recall_test:.4f}")
    print(f"test F1 Score:  {f1_test:.4f}")
    print("\n XGBoost test classification report:")
    print(classification_report(y_val, y_test_pred, digits=4))
    print("-"*50)

Train Accuracy: 0.9645, test Accuracy: 0.7400
test Accuracy:  0.7400
test Precision: 0.7571
test Recall:    0.7192
test F1 Score:  0.7211

 XGBoost test classification report:
              precision    recall  f1-score   support

           0     0.8000    0.5455    0.6486        22
           1     0.7143    0.8929    0.7937        28

    accuracy                         0.7400        50
   macro avg     0.7571    0.7192    0.7211        50
weighted avg     0.7520    0.7400    0.7298        50

--------------------------------------------------
Train Accuracy: 0.9645, test Accuracy: 0.8200
test Accuracy:  0.8200
test Precision: 0.8172
test Recall:    0.8198
test F1 Score:  0.8182

 XGBoost test classification report:
              precision    recall  f1-score   support

           0     0.7826    0.8182    0.8000        22
           1     0.8519    0.8214    0.8364        28

    accuracy                         0.8200        50
   macro avg     0.8172    0.8198    0.8182        5

In [6]:
from sklearn.model_selection import train_test_split

model = xgb.XGBClassifier(
        objective='binary:logistic',
            max_depth=5,
            learning_rate=0.03983915420249222,
            n_estimators=80,
            eval_metric='logloss',
            random_state=42,
        # min_child_weight=5,         
            subsample=0.8,             
        # # colsample_bytree=0.7,       
        
        n_jobs=-1
    )
model.fit(X, y)  
#best_params: {'n_estimators': 80, 'max_depth': 5, 'learning_rate': 0.03983915420249222}
y_test_pred = model.predict(X_test)

acc_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred, average='macro')
recall_test = recall_score(y_test, y_test_pred, average='macro')
f1_test = f1_score(y_test, y_test_pred, average='macro')

train_pred = model.predict(X)  
train_acc = accuracy_score(y, train_pred)
print(f"Train Accuracy: {train_acc:.4f}, test Accuracy: {acc_test:.4f}")

print(f"test Accuracy:  {acc_test:.4f}")
print(f"test Precision: {precision_test:.4f}")
print(f"test Recall:    {recall_test:.4f}")
print(f"test F1 Score:  {f1_test:.4f}")
print("\n XGBoost test classification report:")
print(classification_report(y_test, y_test_pred, digits=4))
print("-"*50)



Train Accuracy: 0.9636, test Accuracy: 0.8226
test Accuracy:  0.8226
test Precision: 0.8193
test Recall:    0.8217
test F1 Score:  0.8203

 XGBoost test classification report:
              precision    recall  f1-score   support

           0     0.7857    0.8148    0.8000        27
           1     0.8529    0.8286    0.8406        35

    accuracy                         0.8226        62
   macro avg     0.8193    0.8217    0.8203        62
weighted avg     0.8237    0.8226    0.8229        62

--------------------------------------------------


In [7]:
y_score_xgb = model.predict_proba(X_test)[:, 1]
df_scores = pd.read_csv("roc_scores_rf.csv")


df_scores["xgb"] = y_score_xgb


df_scores.to_csv("roc_scores_rf.csv", index=False)
